In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import os

print("Libraries imported successfully!")

Libraries imported successfully!


**DATA BREAKDOWN**

Columns names for Plant_1_Generation_Data.csv is "DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD"

Columns names for Plant_1_Weather_Sensor_Data.csv is DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION

In [ ]:
import pandas as pd

# Load datasets
gen_df = pd.read_csv('Plant_1_Generation_Data.csv')
weather_df = pd.read_csv('Plant_1_Weather_Sensor_Data.csv')

In [ ]:
print("Generation Data:")
print(gen_df.info())
print(gen_df.head())

print("\nWeather Data:")
print(weather_df.info())
print(weather_df.head())

Generation Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68778 entries, 0 to 68777
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATE_TIME    68778 non-null  object 
 1   PLANT_ID     68778 non-null  int64  
 2   SOURCE_KEY   68778 non-null  object 
 3   DC_POWER     68778 non-null  float64
 4   AC_POWER     68778 non-null  float64
 5   DAILY_YIELD  68778 non-null  float64
 6   TOTAL_YIELD  68778 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 3.7+ MB
None
          DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0  15-05-2020 00:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
1  15-05-2020 00:00   4135001  1IF53ai7Xc0U56Y       0.0       0.0   
2  15-05-2020 00:00   4135001  3PZuoBAID5Wc2HD       0.0       0.0   
3  15-05-2020 00:00   4135001  7JYdWkrLSPkdwr4       0.0       0.0   
4  15-05-2020 00:00   4135001  McdE0feGgRqW7Ca       0.0       0.0   

   DAILY_

In [ ]:
# Convert to datetime and sort
gen_df['DATE_TIME'] = pd.to_datetime(gen_df['DATE_TIME'], format='%d-%m-%Y %H:%M')
weather_df['DATE_TIME'] = pd.to_datetime(weather_df['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

gen_df = gen_df.sort_values('DATE_TIME')
weather_df = weather_df.sort_values('DATE_TIME')

In [ ]:
# Check NaNs
print("Generation Data Missing Values:\n", gen_df.isnull().sum())
print("\nWeather Data Missing Values:\n", weather_df.isnull().sum())

# Forward-fill or interpolate weather data (if gaps are small)
weather_df = weather_df.fillna(method='ffill')

Generation Data Missing Values:
 DATE_TIME      0
PLANT_ID       0
SOURCE_KEY     0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
TOTAL_YIELD    0
dtype: int64

Weather Data Missing Values:
 DATE_TIME              0
PLANT_ID               0
SOURCE_KEY             0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64


<ipython-input-47-a61e876b3c79>:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_df = weather_df.fillna(method='ffill')


In [ ]:
# Group by time to get plant-level generation stats
agg_gen = gen_df.groupby('DATE_TIME').agg(
    DC_POWER=('DC_POWER', 'sum'),
    AC_POWER=('AC_POWER', 'sum'),
    DAILY_YIELD=('DAILY_YIELD', 'max'),  # Daily yield is cumulative
    TOTAL_YIELD=('TOTAL_YIELD', 'max')   # Total yield is cumulative
).reset_index()

In [ ]:
merged_df = pd.merge_asof(
    left=agg_gen,
    right=weather_df[['DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']],
    on='DATE_TIME',
    direction='nearest'
)

In [ ]:
# Time features
merged_df['HOUR'] = merged_df['DATE_TIME'].dt.hour
merged_df['DAY_OF_WEEK'] = merged_df['DATE_TIME'].dt.dayofweek

# Efficiency: AC/DC conversion ratio
merged_df['CONVERSION_EFFICIENCY'] = merged_df['AC_POWER'] / merged_df['DC_POWER']

# Energy loss due to heat
merged_df['TEMP_DIFF'] = merged_df['MODULE_TEMPERATURE'] - merged_df['AMBIENT_TEMPERATURE']

In [ ]:
merged_df.to_csv('Preprocessed_SolarPlant_Data.csv', index=False)

In [ ]:
# When reading the preprocessed CSV
df = pd.read_csv(
    'Preprocessed_SolarPlant_Data.csv',
    parse_dates=['DATE_TIME'],  # Use correct column name
    index_col='DATE_TIME'        # Set index to existing column
)

# When working with the DataFrame
print(df.index)  # Now properly uses DATE_TIME as datetime index

DatetimeIndex(['2020-05-15 00:00:00', '2020-05-15 00:15:00',
               '2020-05-15 00:30:00', '2020-05-15 00:45:00',
               '2020-05-15 01:00:00', '2020-05-15 01:15:00',
               '2020-05-15 01:30:00', '2020-05-15 01:45:00',
               '2020-05-15 02:00:00', '2020-05-15 02:15:00',
               ...
               '2020-06-17 21:30:00', '2020-06-17 21:45:00',
               '2020-06-17 22:00:00', '2020-06-17 22:15:00',
               '2020-06-17 22:30:00', '2020-06-17 22:45:00',
               '2020-06-17 23:00:00', '2020-06-17 23:15:00',
               '2020-06-17 23:30:00', '2020-06-17 23:45:00'],
              dtype='datetime64[ns]', name='DATE_TIME', length=3158, freq=None)


In [ ]:
merged_df = merged_df.rename(columns={'date_time': 'DATE_TIME'})  # If you accidentally created lowercase version

# Or better: maintain original case throughout
merged_df['DATE_TIME'] = pd.to_datetime(merged_df['DATE_TIME'])  # No change needed

In [ ]:
print("Final Columns:", merged_df.columns.tolist())
# Should output:
# ['DATE_TIME', 'DC_POWER', ... , 'TEMP_DIFF']

Final Columns: ['DATE_TIME', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'HOUR', 'DAY_OF_WEEK', 'CONVERSION_EFFICIENCY', 'TEMP_DIFF']
